## Random bidding strategy 2: max. bid price as bid price boundaries

In [10]:
#Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

#Import validation and testing data sets
rtb_train = pd.read_csv("train.csv")
rtb_train_cropped = rtb_train[['payprice','click','bidprice']]
rtb_validation = pd.read_csv("validation.csv")
rtb_validation_cropped = rtb_validation[['payprice','click']]

In [11]:
#Determine bid prices boundaries with Standard deviation intervalls
meanPayPrice = rtb_validation_cropped['payprice'].mean()
payPriceStd = rtb_validation_cropped['payprice'].std()
print(meanPayPrice)
print(payPriceStd)
maxPrice = (rtb_train_cropped.loc[:,"payprice"].max())
minPrice = (rtb_train_cropped.loc[:,"payprice"].min())


78.23400509994242
60.026062850969545


In [ ]:
#Starting point is the mean of the clicked impressions pay price
constantPriceUp = 0
constantPriceDown = 50
bidPriceListUpperBound = []
bidPriceListLowerBound = []
count = 0

#Determine upper bid price boundaries for the constant bid range
while (count < 300):
    constantPriceUp +=1
    bidPriceListUpperBound.append(constantPriceUp)
    constantPriceDown +=1
    bidPriceListLowerBound.append(constantPriceDown)
    count = count +1
    
numberBids = []
wonBids = []
CTR = [] 
numberClicks = [] 
CPC = [] 
CPM = []
eCPC = []
spentBudget = []

#Repeating bidding in validation set j times
for j in range (0,len(bidPriceListUpperBound)):
    budget = 6250
    bids = 0
    impressions = 0
    clicks = 0
    
    #Creates an array of random bid prices in the boundaries declared above
    randomBids = []
    for i in range(0,len(rtb_validation)):
        bid = random.randrange(0,bidPriceListUpperBound[j],1)
        randomBids.append(bid)
    
    #rtb_validation['random_bids'] = randomBids
    
    for i in range(0,len(rtb_validation_cropped)):
        if budget > 0 and budget > (rtb_validation_cropped['payprice'].values[i]/1000):
            impressions += 1
            if (randomBids[i]/1000) > (rtb_validation_cropped['payprice'].values[i]/(1000)):
                bids += 1
                clicks += rtb_validation_cropped['click'].values[i]
                budget = budget - (rtb_validation_cropped['payprice'].values[i]/1000)
        else: 
            print("Budget is spent!")
            break
    consumedBudget = 6250 - budget
    if (bids != 0):
        ctr = (clicks/bids)*100
        cpm = ((consumedBudget)/bids)*1000
        CTR.append(ctr)
        CPM.append(cpm)
        
    else:
        ctr = 0
        cpm = 0
        CTR.append(ctr)
        CPM.append(cpm)
        
    if (clicks != 0):
        cpc = (consumedBudget)/clicks
        CPC.append(cpc)
        
    else:
        cpc = 0
        CPC.append(cpc)
        
    #Output for the alrogithm
    print(bidPriceListUpperBound[j])
    numberBids.append(impressions)
    #print(numberBids)
    wonBids.append(bids)
    #print(wonBids)
    #print(CTR)
    numberClicks.append(clicks)
    #print(numberClicks)
    #print(CPC)
    #print(CPM)
    spentBudget.append(consumedBudget)    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
Budget is spent!
135
Budget is spent!
136
Budget is spent!
137
Budget is spent!
138
Budget is spent!
139
Budget is spent!
140
Budget is spent!
141
Budget is spent!
142
Budget is spent!
143
Budget is spent!
144
Budget is spent!
145
Budget is spent!
146
Budget is spent!
147
Budget is spent!
148
Budget is spent!
149
Budget is spent!
150
Budget is spent!
151
Budget is spent!
152
Budget is spent!
153
Budget is spent!
154
Budget is spent!
155
Budget is spent!
156
Budget is spent!
157
Budget is spent!
158
Budget is spent!
159
Budget is spent!
160
Budget is spent!
161
Budge

In [ ]:
#Store summary statistics in a dataframe and plot graphs for each of them
summaryStatistics = pd.DataFrame({
    'Constant bid price': bidPriceListUpperBound,
    'numberBids': numberBids,
    'wonBids': wonBids,
    'CTR': CTR,
    'Clicks': numberClicks,
    'eCPC': CPC, 
    'CPM': CPM,
    'Budget spent': spentBudget,
})
print(summaryStatistics)

In [ ]:
summaryStatistics.plot(x='Constant bid price', y='numberBids', style='o'); summaryStatistics.plot(x='Constant bid price', y='wonBids', style='o')
summaryStatistics.plot(x='Constant bid price', y='CTR', style='o')
summaryStatistics.plot(x='Constant bid price', y='Clicks', style='o')
summaryStatistics.plot(x='Constant bid price', y='eCPC', style='o')
summaryStatistics.plot(x='Constant bid price', y='CPM', style='o')
summaryStatistics.plot(x='Constant bid price', y='Budget spent', color = "black")

In [ ]:
# VALIDATION SET - Budget of 6250
# plt.figure()
fig, axes = plt.subplots(nrows= 2, ncols = 3, figsize=(15,7) )

ax1= summaryStatistics.plot(ax=axes[0,0],x='Constant bid price', y='Clicks',color ='black', alpha = 0.8)
ax1.set_ylabel('# of Clicks')
ax1.set_xlabel('Upper Bid Price Boundary [in Fen]')
ax1.set_xticks(np.arange(0,305,50))
ax1.set_yticks(np.arange(0,100,10))
ax1.set_ylim(top=80)
# ymax = summaryStatistics['Clicks'].max()
# xpos = numberClicks.index(ymax)
# xmax = summaryStatistics.iloc[xpos]['Constant bid price']
# ax1.annotate('local max', xy=(80,68), xytext=(80,73),
# arrowprops=dict(facecolor='black', shrink=0.05),)

ax2= summaryStatistics.plot(ax=axes[0,1],x='Constant bid price', y='CPM',color ='black', alpha = 0.8)
ax2.set_ylabel('CPM [Fen]')
ax2.set_xlabel('Upper Bid Price Boundary [in Fen]')
ax2.set_xticks(np.arange(0,305,50))
ax2.set_yticks(np.arange(0,100,10))
ax2.set_ylim(top=90)

ax3= summaryStatistics.plot(ax=axes[0,2],x='Constant bid price', y='CTR',color ='black', alpha = 0.8)
ax3.set_ylabel('CTR [in %]')
ax3.set_xlabel('Upper Bid Price Boundary [in Fen]')
ax3.set_xticks(np.arange(0,305,50))
ax3.set_yticks(np.arange(0,0.1,0.01))  #ax3.autoscale(enable=True, axis='y', tight=True)
ax3.set_ylim(top=0.08)

ax4= summaryStatistics.plot(ax=axes[1,0],x='Constant bid price', y='wonBids',color ='black', alpha = 0.8)
ax4.set_ylabel('# of Bids won')
ax4.set_xlabel('Upper Bid Price Boundary [in Fen]')
ax4.set_xticks(np.arange(0,305,50))
ax4.set_yticks(np.arange(0,190000,20000))
ax4.set_ylim(top=180000)

ax5= summaryStatistics.plot(ax=axes[1,1],x='Constant bid price', y='numberBids',color ='black', alpha = 0.8)
ax5.set_ylabel('# of Bids')
ax5.set_xlabel('Upper Bid Price Boundary [in Fen]')
ax5.set_xticks(np.arange(0,305,50))
ax5.set_yticks(np.arange(0,1000000,50000))  #ax3.autoscale(enable=True, axis='y', tight=True)
ax5.set_ylim(top=350000)

ax6= summaryStatistics.plot(ax=axes[1,2],x='Constant bid price', y='eCPC',color ='black', alpha = 0.8)
ax6.set_ylabel('eCPC [FEN]')
ax6.set_xlabel('Upper Bid Price Boundary [in Fen]')
ax6.set_xticks(np.arange(0,305,50))
ax6.set_yticks(np.arange(0,210,20))  #ax3.autoscale(enable=True, axis='y', tight=True)
ax6.set_ylim(top=160)

plt.tight_layout(h_pad = 0.2)

#Save image
plt.savefig('numberBidsRandom2.pdf',bbox_inches='tight')